In [1]:
from libs import cloudflare_r2_service
from tqdm import tqdm
from tqdm.asyncio import tqdm as async_tqdm
import os
import json
import asyncio

IMAGES_FOLDER = "NoobAI-XL-Vpred-v1.0"
R2_PRIFIX = "NoobAI-XL-Vpred-v1.0"
CACHE_CONTROL = "public, max-age=31536000, immutable"

In [2]:
# 读取生成的图片的配置文件
working_dir = os.path.join("tmp", IMAGES_FOLDER)
with open(os.path.join(working_dir, "image_data.json"), "r") as f:
    image_data: list[dict] = json.load(f)

In [3]:
# image_urls: list[str | None] = [] * len(image_data)
# for data in tqdm(image_data):
#     image_urls.append(
#         cloudflare_r2_service.upload_file_to_r2(
#             file_path=os.path.join(working_dir, f"{data["index"]}.webp"),
#             path_prefix=IMAGES_FOLDER,
#         )
#     )

In [4]:
image_urls: list[str | None] = [None] * len(image_data)
semaphore = asyncio.Semaphore(128)
upload_tasks = [
    cloudflare_r2_service.aio_upload_file_to_r2(
        semaphore=semaphore,
        file_path=os.path.join(working_dir, "images", f"{data['index']}.webp"),
        index=i,
        path_prefix=R2_PRIFIX,
        cache_control=CACHE_CONTROL,
    )
    for i, data in enumerate(image_data)
]
for coro in async_tqdm(
    asyncio.as_completed(upload_tasks), total=len(upload_tasks), desc="Uploading files"
):
    result, index = await coro
    image_urls[index] = result

Uploading files: 100%|██████████| 2592/2592 [01:02<00:00, 41.26it/s]


In [5]:
# 保存上传完成后的信息
for i in range(len(image_data)):
    image_data[i]["image_url"] = image_urls[i]
with open(os.path.join(working_dir, "image_data.json"), "w") as f:
    json.dump(image_data, f, indent=4, ensure_ascii=False)